<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import math

def log2int(x):
  return(math.frexp(x)[1] - 1)

def length(n):
  return(log2int(n+1))

def index(n,m):
  a = (n+1)%(2**(m+1))
  b = 2**m
  if a < b:
    return(0)
  else:
    return(1)

def convert_numeral_to_onesharp(x):
   if x == 0:
     return('#')
   else:
     return('1')    

def s(n):
  k = length(n)  
  s = [convert_numeral_to_onesharp(index(n,k-i-1)) for i in range(k)] 
  t = "".join(s) 
  return(t)

n = 102
k = length(n)
print(k)
b = [convert_numeral_to_onesharp(index(n,k-i-1)) for i in range(n)]    
''.join(b)

for n in range(256):
  print((n,s(n)))
 

6
(0, '')
(1, '#')
(2, '1')
(3, '##')
(4, '#1')
(5, '1#')
(6, '11')
(7, '###')
(8, '##1')
(9, '#1#')
(10, '#11')
(11, '1##')
(12, '1#1')
(13, '11#')
(14, '111')
(15, '####')
(16, '###1')
(17, '##1#')
(18, '##11')
(19, '#1##')
(20, '#1#1')
(21, '#11#')
(22, '#111')
(23, '1###')
(24, '1##1')
(25, '1#1#')
(26, '1#11')
(27, '11##')
(28, '11#1')
(29, '111#')
(30, '1111')
(31, '#####')
(32, '####1')
(33, '###1#')
(34, '###11')
(35, '##1##')
(36, '##1#1')
(37, '##11#')
(38, '##111')
(39, '#1###')
(40, '#1##1')
(41, '#1#1#')
(42, '#1#11')
(43, '#11##')
(44, '#11#1')
(45, '#111#')
(46, '#1111')
(47, '1####')
(48, '1###1')
(49, '1##1#')
(50, '1##11')
(51, '1#1##')
(52, '1#1#1')
(53, '1#11#')
(54, '1#111')
(55, '11###')
(56, '11##1')
(57, '11#1#')
(58, '11#11')
(59, '111##')
(60, '111#1')
(61, '1111#')
(62, '11111')
(63, '######')
(64, '#####1')
(65, '####1#')
(66, '####11')
(67, '###1##')
(68, '###1#1')
(69, '###11#')
(70, '###111')
(71, '##1###')
(72, '##1##1')
(73, '##1#1#')
(74, '##1#11')
(75

In [43]:
s(55)

'11###'

In [21]:
n = 101
k = length(n)
for i in range(k):
  print(i,index(n,i))

0 0
1 1
2 1
3 0
4 0
5 1


In [22]:
n = 102
k = length(n)
for i in range(k):
  print(i,index(n,i))

0 1
1 1
2 1
3 0
4 0
5 1


6


NameError: ignored